Initialisation:

Assume: $\mathcal{U} = 0.05, \mu = 0.0005, f = 0.156, \rho = 0.0005, \gamma = 0.1$ 

Infer: $s = \frac{\mathcal{U}}{1 - \mathcal{U}}\times f - \mu$ and $r = \rho + \mu$

Assume: $\pi_1 = 0.5$

Infer: $\frac{\pi_0}{w^*} \in [0,1 - \pi_1]$

In [2]:
u = 0.05 #unemployment rate
mu = 0.0005 #working population renewal rate
f = 0.156 #job finding rate
rho = 0.0005 #time discount rate
g = 0.1 #worker bargaining power
s = (u/(1 - u))*f - mu #inferred job separation rate
r = mu + rho #composite discount rate
pi1 = 0.5
pi0_wstMin = 0
pi0_wstMax = 1 - pi1

0.5

Wage-benefit sensitivity for job movers:

Change in $\pi_0$:
$$\left.\frac{dw_n}{d\pi_0}\right|_{\text{JM}} = \frac{(1 - \gamma)(r + s)}{r + \gamma f + s\pi_1} \times \frac{r + \gamma f}{r + \gamma f + s}$$

Change in $\pi_1$:
$$\left.\frac{dw_n}{w_{n-1} d\pi_1}\right|_{\text{JM, } w_{n-1} = w^*} = \frac{(1 - \gamma)(r + s)}{r + \gamma f + s\pi_1} \times \frac{r + \gamma f}{r + \gamma f + s\pi_1} \times \left[1 -  \frac{s}{r + \gamma f + s} \times \frac{\pi_0}{w^*} \right]
$$

Wage-benefit sensitivity for eligible job finders if the reform does not apply to their contemporaneous benefits:

Change in $\pi_0$:
$$
\left.\frac{dw_n}{d\pi_0}\right|_{\text{EJF}} = - \frac{(1 - \gamma)(r + s)}{r + \gamma f + s\pi_1} \times \frac{s}{r + \gamma f + s}
$$

Change in $\pi_1$:

$$
\left.\frac{dw_n}{w_{n-1} d\pi_1}\right|_{\text{EJF, } w_{n-1} = w^*} = - \frac{(1 - \gamma)(r + s)}{r + \gamma f + s\pi_1} \times \frac{s}{r + \gamma f + s\pi_1} \times \left[ \pi_1  + \frac{r + \gamma f}{r + \gamma f + s} \times \frac{\pi_0}{w^*} \right]
$$


In [3]:
function dwdb_pi0_JM(; g=g, r=r, s=s, f=f, pi1 = pi1)
    return (((1 - g)*(r + s))/(r + g*f + s))*((r + g*f)/(r + g*f + s*pi1))
end


function dwdb_pi1_JM(pi0_wst; g=g, r=r, s=s, f=f, pi1 = pi1)
    return (((1 - g)*(r + s))/(r + g*f + s*pi1))*((r + g*f)/(r + g*f + s*pi1))*(1 - ((s)/(r + g*f + s))*pi0_wst)
end


function dwdb_pi0_EJF(; g=g, r=r, s=s, f=f, pi1 = pi1)
    return -(((1 - g)*(r + s))/(r + g*f + s*pi))*((s)/(r + g*f + s))
end


function dwdb_pi1_EJF(pi0_wst; g=g, r=r, s=s, f=f, pi1 = pi1)
    return -(((1 - g)*(r + s))/(r + g*f + s*pi1))*((s)/(r + g*f + s*pi1))*(pi1 + ((r + g*f)/(r + g*f + s))*pi0_wst)
end




dwdb_pi1_EJF (generic function with 1 method)

The mass of job stayers who are not employed for the first time ($n > 0$) $\delta t$ months after the policy change:

$$
\omega_{(\text{JS,} n > 0)} = (\mathcal{E} - \mathcal{E}_0) \times \exp(-s \delta t) \times \exp(-\mu \delta t) 
$$
where $\mathcal{E}$ ($= \frac{f}{\mu + f + s}$) is the mass of employed workers and $\mathcal{E}_0$ ($= \frac{\mu}{\mu + f }\times \frac{f}{\mu + s} $) is the mass of workers who are employed for the first time.



The mass of job movers $\delta t$ months after the policy change:

$$
\omega_{(\text{JM})} = \mathcal{E} \times \left[ (1 - \exp(-s\delta t)) - s \times \frac{\exp(-s \delta t) - \exp(-f\delta t)}{f - s} \right] \times \exp(- \mu \delta t) 
$$


The mass of eligible job finders $\delta t$ months after the policy change:

$$
\omega_{(\text{EJF})} = (\mathcal{U} - \mathcal{U}_0) \times (1 - \exp(-f\delta t)) \times \exp(- \mu \delta t)
$$
where $\mathcal{U}$ ($= \frac{\mu + s}{\mu + f + s}$) is the mass of unemployed workers and $\mathcal{U}_0$ ($ = \frac{\mu}{\mu + f}$) is the mass of ineligible (unemployed) workers.




In [4]:
function omega_JS(dt;mu = mu, f=f, s=s)
    return (((f)/(mu + f + s)) - ((mu*f)/((mu + f)*(mu + s))))*(exp(-s*dt))*exp(-mu*dt)
end

function omega_JM(dt;mu = mu, f=f, s=s)
    return ((f)/(mu + f + s))*(1 - exp(-s*dt) - s*((exp(-s*dt) - exp(-f*dt))/(f - s)))*exp(-mu*dt)
end

function omega_EJF(dt;mu = mu, f=f, s=s)
    return (((mu + s)/(mu + f + s)) - ((mu)/(mu + f)))*(1 - exp(-f*dt))*exp(-mu*dt)
end


omega_EJF (generic function with 1 method)

Average predicted wage effect of a change in $\pi_0$ in case the reform applies to the contemporaneous benefits of eligible job finders:

$$
\mathbb{E}\left[\frac{dw_n}{d\pi_0} \right] \approx \frac{  \omega_{(\text{JM})}   + \omega_{(\text{EJF})} }{\omega_{(\text{JS, } n> 0)}  + \omega_{(\text{JM})} + \omega_{(\text{EJF})}  } \times \left.\frac{dw_n}{d\pi_0}\right|_{\text{JM}}
$$

Average predicted wage effect of a change in $\pi_0$ in case the reform does not apply to the contemporaneous benefits of eligible job finders:

$$
\mathbb{E}\left[\frac{dw_n}{d\pi_0} \right] \approx \frac{\omega_{(\text{JM})} \times \left.\frac{dw_n}{d\pi_0}\right|_{\text{JM}}  + \omega_{(\text{EJF})} \times \left.\frac{dw_n}{d\pi_0}\right|_{\text{EJF}}}{\omega_{(\text{JS, } n> 0)}  + \omega_{(\text{JM})} + \omega_{(\text{EJF})}  }
$$

Average predicted wage effect of a change in $\pi_1$ in case the reform applies to the contemporaneous benefits of eligible job finders:

$$
\mathbb{E}\left[\left.\frac{dw_n}{w_{n-1}d\pi_1}\right|_{w_{n-1} = w^*} \right] \approx \frac{  \omega_{(\text{JM})}  + \omega_{(\text{EJF})} }{\omega_{(\text{JS, }n>0)}  + \omega_{(\text{JM})}  + \omega_{(\text{EJF})}  }  \times \left.\frac{dw_n}{w_{n-1}d\pi_1}\right|_{\text{JM, }w_{n-1} = w^*}
$$

Average predicted wage effect of a change in $\pi_1$ in case the reform does not apply to the contemporaneous benefits of eligible job finders:

$$
\mathbb{E}\left[\left.\frac{dw_n}{w_{n-1}d\pi_1}\right|_{w_{n-1} = w^*} \right] \approx \frac{  \omega_{(\text{JM})} \times \left.\frac{dw_n}{w_{n-1}d\pi_1}\right|_{\text{JM, }w_{n-1} = w^*} + \omega_{(\text{EJF})} \times \left.\frac{dw_n}{w_{n-1}d\pi_1}\right|_{\text{EJF, } w_{n-1} = w^*}}{\omega_{(\text{JS, }n>0)}  + \omega_{(\text{JM})}  + \omega_{(\text{EJF})}  } 
$$


In [5]:
function Edwdb_pi0_A(dt)
    return ((omega_JM(dt) + omega_EJF(dt))*dwdb_pi0_JM())/(omega_EJF(dt) + omega_JM(dt) + omega_JS(dt))
end

function Edwdb_pi0_NA(dt)
    return (omega_JM(dt)*dwdb_pi0_JM() + omega_EJF(dt)*dwdb_pi0_EJF())/(omega_EJF(dt) + omega_JM(dt) + omega_JS(dt))
end

function Edwdb_pi1_A(pi0_wst,dt)
    return ((omega_JM(dt) + omega_EJF(dt))/(omega_JM(dt) + omega_EJF(dt) + omega_JS(dt)))*dwdb_pi1_JM(pi0_wst)
end

function Edwdb_pi1_NA(pi0_wst,dt)
    return (omega_JM(dt)*dwdb_pi1_JM(pi0_wst) + omega_EJF(dt)*dwdb_pi1_EJF(pi0_wst))/(omega_EJF(dt) +  omega_JM(dt) + omega_JS(dt) )
end



Edwdb_pi1_NA (generic function with 1 method)

In [8]:
using Plots

DT = collect(0.0:0.01:12.0)

ppi0 = plot(DT, Edwdb_pi0_A.(DT), xticks = [0,1,2,3,4,5,6,7,8,9,10,11,12], yaxis = "average wage-benefit sensitivity", yticks = 0.0:0.005:0.03, ylims = (-0.0025,0.03), xaxis = "time since reform in months", label = "applies to current b of EJF", title = "calibration: \\mu = $(mu), \\rho = $(rho), \\gamma = $(g), \\pi_{1} = $(pi1), f = $(f), s = $(round(s,digits = 4))", width = 2.5, linecolor = :red, dpi = 600, wsize = (600,400), guidefontsize = 9, guidefontvalign = :left, titlefontsize = 11, titlefontvalign = :bottom, guidefonthalign = :left, guide_position = :bottom, legend = :topleft)
plot!(ppi0, DT, Edwdb_pi0_NA.(DT), label = "does not apply to current b of EJF", width = 2.5, linecolor = :blue)
savefig(ppi0,"ppi0")


ppi1 = plot(DT, Edwdb_pi1_A.(pi0_wstMax,DT), xticks = [0,1,2,3,4,5,6,7,8,9,10,11,12], yaxis = "average wage-benefit sensitivity", yticks = 0.0:0.005:0.03, ylims = (-0.0025,0.03), xaxis = "time since reform in months", linestyle = :dot, label = "applies to current b of EJF (min)", title = "calibration: \\mu = $(mu), \\rho = $(rho), \\gamma = $(g), \\pi_{1} = $(pi1), f = $(f), s = $(round(s,digits = 4))", width = 2.5, color = :red, dpi = 600, wsize = (600,400), guidefontsize = 9, guidefontvalign = :left, titlefontsize = 11, titlefontvalign = :bottom, guidefonthalign = :left, guide_position = :bottom, legend= :topleft)
plot!(ppi1, DT, Edwdb_pi1_A.(pi0_wstMin,DT), label = "applies to current b of EJF (max)", width = 2.5, linecolor = :red)
plot!(ppi1, DT, Edwdb_pi1_NA.(pi0_wstMax,DT), label = "does not apply to current b of EJF (min)", width = 2.5, linecolor = :blue, linestyle = :dot)
plot!(ppi1, DT, Edwdb_pi1_NA.(pi0_wstMin,DT), label = "does not apply to current b of EJF (max)", width = 2.5, linecolor = :blue)
savefig(ppi1,"ppi1")

pomega = plot(DT, omega_JS.(DT), xticks = [0,1,2,3,4,5,6,7,8,9,10,11,12], yticks = 0.0:0.1:1.0, xaxis = "time since reform in months", label = "mass of job stayers with n > 0", title = "calibration: \\mu = $(mu), \\rho = $(rho), f = $(f), s = $(round(s,digits = 4))", width = 2.5, linecolor = :green, dpi = 600, wsize = (600,400), guidefontsize = 9, guidefontvalign = :left, titlefontsize = 11, titlefontvalign = :bottom, guidefonthalign = :left, guide_position = :bottom, legend = :left, ylims = (0.0,1.0))
plot!(pomega, DT, omega_JM.(DT), label = "mass of job movers", width = 2.5, linecolor = :purple)
plot!(pomega, DT, omega_EJF.(DT), label = "mass of eligible job finders", width = 2.5, linecolor = :orange)
savefig(pomega,"pomega")


